In [ ]:
using WAV;
using Gen;
using Printf;
import Random;
include("./model.jl");
include("./plotting.jl");
include("./proposals.jl");
include("./gammatonegram.jl");
include("./time_helpers.jl");
include("./inference_helpers.jl");
include("./rendering.jl")
include("./gaussian_helpers.jl")

In [ ]:
steps = Dict("t"=>0.020, "min"=> 0.025, "f"=>4, "scale"=>"ERB"); 

audio_sr = 20000; #check that this is the same as dreaming.jl
f = get_element_gp_freqs(audio_sr, steps)
noise_mu_params = Dict(:sigma=> 10.0, :scale=> 4.0, :epsilon=> 1.0)
prior_cov = generate_covariance_matrix(f, noise_mu_params)

source_params=Dict(
    "types"=>["tone","noise"], #"tone", "noise", "harmonic"
    "n_harmonics"=>30,
    "n_sources"=>Dict("type"=>"max","val"=>5),
    "n_elements"=>Dict("type"=>"max","val"=>7),
    "tp"=>Dict(
        "wait"=>Dict("a"=>Dict("dist"=>gamma,"args"=>(20,1)),
                     "mu"=>Dict("dist"=>log_normal, "args"=>(-2.0, 1.5))),
        "dur_minus_min"=>Dict("a"=>Dict("dist"=>gamma, "args"=>(20, 1)),
                    "mu"=>Dict("dist"=>log_normal, "args"=>(-2.0, 1.5)))

        ),
    "gp"=>Dict(
        "erb"=>Dict("mu"=>Dict("dist"=>uniform,"args"=>(freq_to_ERB(20.0), freq_to_ERB(9999.0))),
                    "scale"=>Dict("dist"=>gamma,"args"=>(0.5,1)),
                    "sigma"=>Dict("dist"=>gamma,"args"=>(3,1)),
                    "epsilon"=>Dict("dist"=>gamma,"args"=>(1,0.1))),
        "amp"=>Dict(
            "1D"=>Dict("mu"=>Dict("dist"=>normal,"args"=>(50, 15)),
                    "scale"=>Dict("dist"=>gamma,"args"=>(0.5,1)),
                    "sigma"=>Dict("dist"=>gamma,"args"=>(7,1)),
                    "epsilon"=>Dict("dist"=>gamma,"args"=>(1,1))),
            "2D"=>Dict("mu"=>Dict("dist"=>mvnormal,"args"=>(fill(10.0,length(f)),prior_cov)),
                    "scale_t"=>Dict("dist"=>gamma,"args"=>(0.5,2)),
                    "scale_f"=>Dict("dist"=>gamma,"args"=>(2,2)),
                    "sigma"=>Dict("dist"=>gamma,"args"=>(10,2)),
                    "epsilon"=>Dict("dist"=>gamma,"args"=>(1,1)))
            )
        ),
    "enable_switch"=>false, #inference move to switch source_types
    "duration_limit"=>2.5, #upper limit on element duration in seconds
    "dream"=>true #generating data or inference?
    )


gtg_params=Dict("twin"=>0.025, "thop"=>0.010, "nfilts"=>64, "fmin"=>20, "width"=>0.50, 
    "log_constant"=>1e-80, "dB_threshold"=>20, "ref"=>1e-6, "plot_max"=>100)

obs_noise = Dict("type"=>"fixed","val"=>1.0)

# p = (source_params, steps, gtg_params, obs_noise)

In [ ]:
#source_params, steps, gtg_params, obs_noise = include("./base_params.jl")
# audio_sr=20000; 
wts, f = gtg_weights(audio_sr, gtg_params)
scene_duration = 2.0
# args = (source_params, float(scene_duration), wts, steps, Int(audio_sr), obs_noise, gtg_params);

In [ ]:
args = (source_params, float(scene_duration), wts, steps, Int(audio_sr), obs_noise, gtg_params)
trace, = generate(generate_scene, args);
scene_gram, t, scene_wave, source_waves, element_waves = get_retval(trace)
wavwrite(scene_wave/maximum(abs.(scene_wave)), "scene.wav", Fs=audio_sr)
plot_gtg(scene_gram, scene_duration, audio_sr/2., 20, 100)
#plot_sources(trace, demo_gram, 0)

In [ ]:
latent=:erb
scene_t = get_element_gp_times([0,scene_duration], steps["t"])
gp_elem = Dict()
gp_elem[:erb] = cat([trace[:source=>1=>(:element,i)=>:erb] for i in 1:trace[:source=>1=>:n_elements]]...,dims=1)
source_latents=Dict()
for k in [:mu, :sigma, :epsilon, :scale]
    println("$k: ", trace[:source=>1=>:erb=>k])
    source_latents[k]=round(trace[:source=>1=>:erb=>k],digits=3)
end
abs_t = absolute_timing(get_submap(get_choices(trace),:source=>1,), steps["min"])
gp_elem[:t] = cat( [get_element_gp_times(tt, steps["t"]) for tt in abs_t ]... ,dims=1)
mask = [t in gp_elem[:t] ? 1.0 : 0.0 for t in scene_t]
embedded_gp = []; k = 1; 
m = mean(gp_elem[latent]); 
for t_idx in 1:length(scene_t)
    if mask[t_idx] == 1
        push!(embedded_gp, gp_elem[latent][k])
        k += 1
    else
        push!(embedded_gp, m)
    end
end
embedded_gp = embedded_gp .- m #should this be scaled? 
plot(embedded_gp)
title(string("mu: ", source_latents[:mu], " sigma: ", source_latents[:sigma], 
        " scale: ", source_latents[:scale], " epsilon: ", source_latents[:epsilon]))
ylim([-10,10]);

a=embedded_gp[1:end-2] 
b=embedded_gp[2:end-1] 
c=embedded_gp[3:end]

z=(a./2 .+ c./2 .- b).^2

m1=mask[1:end-2] .* mask[2:end-1] .* mask[3:end]

x=sum(z.*m1)/sum(m1)
epsstr=source_latents[:epsilon]
println("EPSILon--- Predict: $x, true: $epsstr")

In [ ]:
include("./initialization.jl");
demo_gram, wts, scene_duration, audio_sr = load_sound("1i", gtg_params)
demo_guide = read_guide("1A")
args = (source_params, float(scene_duration), wts, steps, Int(audio_sr), obs_noise, gtg_params)
constraints = choicemap()
constraints[:scene] = demo_gram;
trace = init_from_likelihood(100, demo_guide, demo_gram, args) #: generate(generate_scene, args, constraints);

In [ ]:
n = []
p = "ne" #"ne" or "amp"
st = "noise" #"tone" or "noise" only matters for gps
for i = 1:100
    sd = float(round((2.5-0.5)*rand() + 0.5,digits=3))
    args = (source_params, sd, wts, steps, Int(audio_sr), obs_noise, gtg_params);
    trace, = generate(generate_scene, args);
    if p == "erb" || p == "amp"
        for j = 1:trace[:n_sources]
            if source_params["types"][trace[:source => j => :source_type]] == st
                for k = 1:trace[:source => j => :n_elements]
                    try
                        append!(n, trace[:source => j => (:element, k) => Symbol(p)])
                    catch
                        break
                    end
                end
            end
        end
    elseif p == "ne"
        scene_gram, t, scene_wave, source_waves, element_waves = get_retval(trace)
        n_elements = 0;
        for j = 1:length(element_waves)
            for k = 1:size(element_waves[j])[2]
                element_wave = element_waves[j][:, k]
                element_gram, = gammatonegram(element_wave, wts, audio_sr, gtg_params)
                if any(element_gram .> gtg_params["dB_threshold"])
                    n_elements += 1
                end
            end 
        end
       push!(n, n_elements)
    end
end

In [ ]:
hist(n)
if p == "amp" && st == "noise"
    title(string("Noise amp vals, mu = ", string(source_params["gp"][p]["2D"]["mu"]["args"][1]),"; sig = ", string(source_params["gp"][p]["2D"]["mu"]["args"][2])))
elseif p == "amp" && st == "tone"
    title(string("Tone amp vals, mu = ", string(source_params["gp"][p]["1D"]["mu"]["args"][1]),"; sig = ", string(source_params["gp"][p]["1D"]["mu"]["args"][2])))
elseif p == "erb"
    title("Erb vals")
elseif p == "ne"
    title(string("max_source=" , string(source_params["n_sources"]["val"]), ", max_elem=" , string(source_params["n_elements"]["val"]),". n=", string(length(n))))
end